***(i) Import the function from the module "mzprojection"***

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
# from mzprojection import mzprojection_multivariate, split_long_time_series, calc_correlation
from mzprojection import mzprojection_multivariate_long_time_series, calc_correlation_long_time_series, calc_residual_long_time_series

# help(mzprojection_multivariate_long_time_series)

***(ii) Prepare a long-time-series data***  
Here, instead of an ensemble set of short-time-series data, we use a long time-series data by assuming ergodicity.

In [ ]:
#= Read sample data =
ds = xr.open_dataset('../sample_data/sample_multivariate_L96xy.nc')
t_raw    = np.array(ds.t)      # Time t
u_raw    = np.array(ds.u).T    # Variable of interest u(t)
dudt_raw = np.array(ds.dudt).T # = du/dt
f_raw    = np.array(ds.f).T    # Analyzed data f(t)
print("t_raw[:].shape=",t_raw.shape)
print("u_raw[:,nu].shape=",u_raw.shape)
print("f_raw[:,nf].shape=",f_raw.shape,"# nu/=nf is available.")

fig = plt.figure(figsize=(14,2.5))
ax = fig.add_subplot(111)
ax.plot(t_raw,f_raw[:,0],label="f(t)")
ax.plot(t_raw,u_raw[:,0],label="u(t)")
ax.set_xlim(0,None)
ax.set_xlabel("Time t")
ax.set_ylabel("Long time-series u(t), f(t)")
plt.legend()
plt.show()

***(iii) Apply the Moti-Zwanzig projection operator method***  
The projection of $f(t)^i$ on $u(t)^i$ is calculated as,  
    $f(t)=\Omega u(t)+s(t)+r(t)$,  
    $s(t)=-\int_0^t \Gamma(t) u(t-v)dv$.  
The Markov coefficient $\Omega$, the memory function $\Gamma(t)$ and the uncorrelated term $r(t)$ are obtained as outputs.  
(Some correlations, e.g., $\langle r(t) u \rangle$ are also obtained to check the result.)

In [ ]:
delta_t = t_raw[1] - t_raw[0] # Time step size
ista=1000    # Skip time steps in the beginning
t=t_raw[ista:]
u=u_raw[ista:,:]
dudt=dudt_raw[ista:,:]
f=f_raw[ista:,:]
nperiod=400  # Length of time steps to evaluate the memory function
# omega, memoryf, s, r = mzprojection_multivariate_long_time_series(delta_t, u, dudt, f, nperiod=nperiod, flag_terms=True, flag_debug=True)
omega, memoryf = mzprojection_multivariate_long_time_series(delta_t, u, dudt, f, nperiod=nperiod, flag_terms=False, flag_debug=True)
s_raw, r_raw = calc_residual_long_time_series(delta_t, u_raw, f_raw, omega, memoryf)
s = s_raw[ista:,:]
r = r_raw[ista:,:]
t_cor = delta_t * np.arange(memoryf.shape[0])

***(iv-a) Check results: plot $\Omega$ and $\Gamma(t)$***  

In [ ]:
print("Markov coefficient matrix Omega = ", omega[0,0])

fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_xlabel("Correlation time t")
ax.set_ylabel("Memory function $\Gamma(t)$")
ax.plot(t_cor,memoryf[:,0,0],label="$\Gamma(t)$")
plt.legend()
plt.show()

***(iv-b) Check results: plot time evolution of $f(t)=\Omega u(t)+s(t)+r(t)$***  

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_xlabel(r"Time $t$")
ax.set_ylabel(r"$f_i(t)=\Omega_{ij}u_j(t)+s_i(t)+r_i(t)$")
tsta=30
tend=34
ax.plot(t,f[:,0],label=r"$f_i(t)$")
ax.plot(t,np.dot(u[:,:],omega.T)[:,0],label=r"$\Omega_{ij}u_j(t)$")
ax.plot(t,s[:,0],label=r"$s_i(t)$")
ax.plot(t,r[:,0],label=r"$r_i(t)$")
ax.set_xlim(tsta,tend)
plt.legend()
plt.show()

***(iv-c) Check results: Extended generalized fluctuation-dissipation theorem $\Gamma(t) = \langle r(t) du/dt \rangle / \langle u u \rangle$***  

In [ ]:
uu = calc_correlation_long_time_series(u,u,nperiod)
ududt = calc_correlation_long_time_series(u,dudt,nperiod)
ff = calc_correlation_long_time_series(f,f,nperiod)
fdudt = calc_correlation_long_time_series(f,dudt,nperiod)
fu = calc_correlation_long_time_series(f,u,nperiod)
rr = calc_correlation_long_time_series(r,r,nperiod)
rdudt = calc_correlation_long_time_series(r,dudt,nperiod)
ru = calc_correlation_long_time_series(r,u,nperiod)
uu0_inv = np.linalg.inv(uu[0,:,:])

fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_xlabel("Correlation time t")
ax.set_ylabel(r"$\Gamma_{ij}(t) = \langle r_i(t) du/dt_k \rangle \langle u u \rangle^{-1}_{k,j}$")
ax.plot(t_cor,memoryf[:,0,0],label="Re[$\Gamma_{ij}(t)$]")
ax.plot(t_cor,np.dot(rdudt,uu0_inv)[:,0,0],"--",label=r"Re[$\langle r_i(t) du/dt_k \rangle \langle u u \rangle^{-1}_{k,j}$]",lw=3)
plt.legend()
plt.show()

***(iv-d) Check results: $\langle r(t) u \rangle = 0$***  
Namely, the uncorrelated term $r(t)$ indeed extracts the part of $f(t)$ uncorrelated with $u$. It is checked by $\langle r(t) u \rangle \ll \langle f(t) u \rangle$.

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_xlabel("Correlation time t")
ax.set_ylabel("Correlation$")
ax.plot(t_cor,ru[:,0,0],label=r"$\langle r_i(t) u_j \rangle$")
ax.plot(t_cor,fu[:,0,0],label=r"$\langle f_i(t) u_j \rangle$")
plt.legend()
plt.show()